In [1]:
import os

In [2]:
filename = "geoEurope_202206{}{}.zip"

In [4]:
names = []
for day in range(1,31):
    for hour in range(0,24):
        if day < 10:
            day1 = '0'+ str(day)
        else:
            day1 = str(day)
        if hour < 10:
            hour1 = '0' + str(hour)
        else:
            hour1 = str(hour1)
            
        name = filename.format(day1,hour1)
        names.append(name)

In [5]:
# Used to process zip files
'''
for name in names:
    print("Processing:{}".format(name))
    os.system('unzip {}'.format(name))
    os.system('rm {}'.format(name))'''

'\nfor name in names:\n    print("Processing:{}".format(name))\n    os.system(\'unzip {}\'.format(name))\n    os.system(\'rm {}\'.format(name))'

In [6]:
json_file = "./geoEurope/geoEurope_202206{}{}.json"

In [7]:
json_names = []
for day in range(1,31):
    for hour in range(0,24):
        if day < 10:
            day1 = '0'+ str(day)
        else:
            day1 = str(day)
        if hour < 10:
            hour1 = '0' + str(hour)
        else:
            hour1 = str(hour)
            
        name = json_file.format(day1,hour1)
        json_names.append(name)

In [32]:
header = "id\tuser_id\tuser_name\ttime\ttime_day\ttime_hour\tlocation\tcountry\tcontent\tmention_id\tmention_name\treply_id\n"

In [33]:
with open("processed.tsv",'w') as f:
    f.write(header)

In [34]:
import time

In [ ]:
for json_f in json_names:
    print("Now reading file:", json_f)
    result_json = []
    with open(json_f) as f:
        lines = f.readlines()
        for line in lines:
            result = json.loads(line)
            result_json.append(result)

    for record in result_json:
        try:
            id = ''
            if 'id' not in record:
                continue
            if record['id']:
                id = record['id']

            user_id = ''
            if record['user']['id_str']:
                user_id = record['user']['id_str']
            user_name = ''
            if record['user']['screen_name']:
                user_name = record['user']['screen_name'].replace('\t', '  ')

            ts = record['timestamp_ms']
            timestamp = int(ts) + 3600000
            format_date = time.strftime('%Y-%m-%d %H:%M:%S:{}'.format(timestamp % 1000),
                                        time.gmtime(timestamp / 1000.0))
            day = int(format_date.split(' ')[0].split('-')[2])
            time_hour = int(format_date.split(' ')[1].split(':')[0])
            location = ''
            if record['geo']:
                location = str(record['geo']['coordinates'])

            country = ''
            if record['place']:
                country = record['place']['country']

            content = record['text'].replace('\t', ' ').replace('\n', '  ')
            mention_id = ''
            mention_name = ''
            reply = ''
            if record['in_reply_to_user_id_str']:
                reply = record['in_reply_to_user_id_str']

            if record['entities']['user_mentions']:
                ids = []
                names = []
                for r in record['entities']['user_mentions']:
                    ids.append(r['id_str'])
                    names.append(r['screen_name'])
                mention_id = str(ids)
                mention_name = str(names)

            write = "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(id, user_id, user_name, format_date, day,
                                                                              time_hour, location, country, content,
                                                                              mention_id, mention_name, reply)
            # print(write)
            with open("processed.tsv", 'a') as f:
                f.write(write)
        except Exception:
            print("Meet an error.")
            pass